autoencoder + classifier

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import joblib

# === Load and Preprocess Data ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)

# === Convert to PyTorch Tensors ===
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=128)

# === Autoencoder with Encoder Only for Classification ===
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, latent_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)

class Classifier(nn.Module):
    def __init__(self, latent_dim, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

# === Model Init ===
input_dim = X.shape[1]
latent_dim = 64
num_classes = len(np.unique(y))

encoder = Encoder(input_dim, latent_dim)
decoder = Decoder(latent_dim, input_dim)
classifier = Classifier(latent_dim, num_classes)

# === Joint Optimizer and Losses ===
params = list(encoder.parameters()) + list(decoder.parameters()) + list(classifier.parameters())
optimizer = optim.Adam(params, lr=0.001)
recon_loss_fn = nn.MSELoss()
clf_loss_fn = nn.CrossEntropyLoss()  # Can replace with weighted loss if needed

# === Joint Training Loop ===
print("\n🔁 Joint AE + Classifier Training")
for epoch in range(30):
    encoder.train()
    decoder.train()
    classifier.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()

        encoded = encoder(xb)
        reconstructed = decoder(encoded)
        logits = classifier(encoded)

        loss_recon = recon_loss_fn(reconstructed, xb)
        loss_clf = clf_loss_fn(logits, yb)

        loss = loss_recon + loss_clf  # weight them if needed
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Total Loss: {total_loss / len(train_loader):.4f}")

# === Evaluation ===
encoder.eval()
classifier.eval()
with torch.no_grad():
    encoded_test = encoder(X_test_tensor)
    logits_test = classifier(encoded_test)
    y_pred = torch.argmax(logits_test, dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

label_map = {
    0: 'AMAZON', 1: 'CLOUDFLARE', 2: 'DROPBOX', 3: 'FACEBOOK',
    4: 'GMAIL', 5: 'GOOGLE', 6: 'HTTP', 7: 'HTTP_CONNECT', 8: 'HTTP_PROXY',
    9: 'MICROSOFT', 10: 'MSN', 11: 'SKYPE', 12: 'SSL', 13: 'TWITTER',
    14: 'WINDOWS_UPDATE', 15: 'YAHOO', 16: 'YOUTUBE'
}
print("\n✅ Final Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred, target_names=[label_map[i] for i in sorted(label_map)]))

# === Save Models ===
torch.save(encoder.state_dict(), "encoder_joint.pth")
torch.save(decoder.state_dict(), "decoder_joint.pth")
torch.save(classifier.state_dict(), "classifier_joint.pth")



🔁 Joint AE + Classifier Training
Epoch 1, Total Loss: 1.8860
Epoch 2, Total Loss: 1.5089
Epoch 3, Total Loss: 1.3762
Epoch 4, Total Loss: 1.3050
Epoch 5, Total Loss: 1.2477
Epoch 6, Total Loss: 1.2338
Epoch 7, Total Loss: 1.1931
Epoch 8, Total Loss: 1.1719
Epoch 9, Total Loss: 1.1572
Epoch 10, Total Loss: 1.1418
Epoch 11, Total Loss: 1.1233
Epoch 12, Total Loss: 1.1194
Epoch 13, Total Loss: 1.1078
Epoch 14, Total Loss: 1.0986
Epoch 15, Total Loss: 1.0890
Epoch 16, Total Loss: 1.0806
Epoch 17, Total Loss: 1.0805
Epoch 18, Total Loss: 1.0672
Epoch 19, Total Loss: 1.0554
Epoch 20, Total Loss: 1.0651
Epoch 21, Total Loss: 1.0507
Epoch 22, Total Loss: 1.0372
Epoch 23, Total Loss: 1.0392
Epoch 24, Total Loss: 1.0333
Epoch 25, Total Loss: 1.0355
Epoch 26, Total Loss: 1.0226
Epoch 27, Total Loss: 1.0253
Epoch 28, Total Loss: 1.0149
Epoch 29, Total Loss: 1.0067
Epoch 30, Total Loss: 1.0160

✅ Final Accuracy: 0.9350118835412953

📊 Classification Report:
                 precision    recall  f1-

Autoencoder + LSTM

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import joblib

# === Load and Preprocess Data ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=128)

# === Autoencoder + LSTM Classifier ===
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, latent_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)

class LSTMClassifier(nn.Module):
    def __init__(self, latent_dim, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(latent_dim, 128, batch_first=True, num_layers=2, dropout=0.3)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Convert to (batch_size, seq_len=1, latent_dim)
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1])  # Use last hidden state

# === Model Init ===
input_dim = X.shape[1]
latent_dim = 64
num_classes = len(np.unique(y))

encoder = Encoder(input_dim, latent_dim)
decoder = Decoder(latent_dim, input_dim)
classifier = LSTMClassifier(latent_dim, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = encoder.to(device)
decoder = decoder.to(device)
classifier = classifier.to(device)

# === Joint Optimizer and Loss ===
params = list(encoder.parameters()) + list(decoder.parameters()) + list(classifier.parameters())
optimizer = optim.Adam(params, lr=0.001)
recon_loss_fn = nn.MSELoss()
clf_loss_fn = nn.CrossEntropyLoss()

# === Joint Training Loop ===
print("\n🔁 Joint AE + LSTM Training")
for epoch in range(30):
    encoder.train()
    decoder.train()
    classifier.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()

        encoded = encoder(xb)                       # (batch, latent_dim)
        reconstructed = decoder(encoded)           # (batch, input_dim)
        logits = classifier(encoded)               # (batch, num_classes)

        loss_recon = recon_loss_fn(reconstructed, xb)
        loss_clf = clf_loss_fn(logits, yb)
        loss = loss_recon + loss_clf

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Total Loss: {total_loss / len(train_loader):.4f}")

# === Evaluation ===
encoder.eval()
classifier.eval()
with torch.no_grad():
    encoded_test = encoder(X_test_tensor.to(device))
    logits_test = classifier(encoded_test)
    y_pred = torch.argmax(logits_test, dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

label_map = {
    0: 'AMAZON', 1: 'CLOUDFLARE', 2: 'DROPBOX', 3: 'FACEBOOK',
    4: 'GMAIL', 5: 'GOOGLE', 6: 'HTTP', 7: 'HTTP_CONNECT', 8: 'HTTP_PROXY',
    9: 'MICROSOFT', 10: 'MSN', 11: 'SKYPE', 12: 'SSL', 13: 'TWITTER',
    14: 'WINDOWS_UPDATE', 15: 'YAHOO', 16: 'YOUTUBE'
}

print("\n✅ Final Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred, target_names=[label_map[i] for i in sorted(label_map)]))

# === Save Models ===
torch.save(encoder.state_dict(), "encoder_lstm_joint.pth")
torch.save(decoder.state_dict(), "decoder_lstm_joint.pth")
torch.save(classifier.state_dict(), "classifier_lstm_joint.pth")



🔁 Joint AE + LSTM Training
Epoch 1, Total Loss: 2.0205
Epoch 2, Total Loss: 1.6445
Epoch 3, Total Loss: 1.5618
Epoch 4, Total Loss: 1.5220
Epoch 5, Total Loss: 1.4792
Epoch 6, Total Loss: 1.4563
Epoch 7, Total Loss: 1.4266
Epoch 8, Total Loss: 1.4158
Epoch 9, Total Loss: 1.3934
Epoch 10, Total Loss: 1.3803
Epoch 11, Total Loss: 1.3643
Epoch 12, Total Loss: 1.3437
Epoch 13, Total Loss: 1.3390
Epoch 14, Total Loss: 1.3021
Epoch 15, Total Loss: 1.2959
Epoch 16, Total Loss: 1.2551
Epoch 17, Total Loss: 1.2409
Epoch 18, Total Loss: 1.2275
Epoch 19, Total Loss: 1.1927
Epoch 20, Total Loss: 1.1882
Epoch 21, Total Loss: 1.1697
Epoch 22, Total Loss: 1.1566
Epoch 23, Total Loss: 1.1442
Epoch 24, Total Loss: 1.1312
Epoch 25, Total Loss: 1.1289
Epoch 26, Total Loss: 1.1233
Epoch 27, Total Loss: 1.1082
Epoch 28, Total Loss: 1.0966
Epoch 29, Total Loss: 1.1018
Epoch 30, Total Loss: 1.0916

✅ Final Accuracy: 0.8886660724896019

📊 Classification Report:
                 precision    recall  f1-score 

gans + cnn

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib

# === Load Data ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

# === Preprocessing ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=128)

input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
label_smooth_real = 0.9

# === One-hot encoder ===
def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

# === Focal Loss ===
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Generator ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

# === Discriminator ===
class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        return self.model(x)

# === Classifier ===
class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# === Init Models ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G = Generator(latent_dim, num_classes, input_dim).to(device)
D = Discriminator(input_dim, num_classes).to(device)
clf = CNNClassifier(input_dim, num_classes).to(device)

# === Losses and Optimizers ===
bce = nn.BCELoss()
focal_loss = FocalLoss()
opt_G = optim.Adam(G.parameters(), lr=0.0002)
opt_D = optim.Adam(D.parameters(), lr=0.0002)
opt_C = optim.Adam(clf.parameters(), lr=0.001)

# Schedulers
scheduler_C = optim.lr_scheduler.StepLR(opt_C, step_size=15, gamma=0.5)

# === Training Loop ===
epochs = 100
for epoch in range(epochs):
    G.train(); D.train(); clf.train()
    total_d_loss, total_g_loss, total_c_loss = 0, 0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        batch_size = xb.size(0)

        # === Train Discriminator ===
        real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((batch_size, 1)).to(device)

        z = torch.randn(batch_size, latent_dim).to(device)
        fake_data = G(z, yb)

        D_real = D(xb, yb)
        D_fake = D(fake_data.detach(), yb)

        loss_D = bce(D_real, real_labels) + bce(D_fake, fake_labels)
        opt_D.zero_grad()
        loss_D.backward()
        opt_D.step()

        # === Train Generator ===
        D_fake = D(fake_data, yb)
        loss_G = bce(D_fake, real_labels)
        opt_G.zero_grad()
        loss_G.backward()
        opt_G.step()

        # === Train Classifier on real + fake ===
        synthetic_data = G(torch.randn(batch_size, latent_dim).to(device), yb)
        combined_data = torch.cat([xb, synthetic_data], dim=0)
        combined_labels = torch.cat([yb, yb], dim=0)

        preds = clf(combined_data)
        loss_C = focal_loss(preds, combined_labels)
        opt_C.zero_grad()
        loss_C.backward()
        opt_C.step()

        total_d_loss += loss_D.item()
        total_g_loss += loss_G.item()
        total_c_loss += loss_C.item()

    scheduler_C.step()
    print(f"[Epoch {epoch+1}] D Loss: {total_d_loss:.4f}, G Loss: {total_g_loss:.4f}, C Loss: {total_c_loss:.4f}")

# === Evaluation ===
clf.eval()
with torch.no_grad():
    preds = clf(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

label_map = {
    0: 'AMAZON', 1: 'CLOUDFLARE', 2: 'DROPBOX', 3: 'FACEBOOK',
    4: 'GMAIL', 5: 'GOOGLE', 6: 'HTTP', 7: 'HTTP_CONNECT', 8: 'HTTP_PROXY',
    9: 'MICROSOFT', 10: 'MSN', 11: 'SKYPE', 12: 'SSL', 13: 'TWITTER',
    14: 'WINDOWS_UPDATE', 15: 'YAHOO', 16: 'YOUTUBE'
}
print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred, target_names=[label_map[i] for i in sorted(label_map)]))

# === Save Models ===
torch.save(G.state_dict(), "generator_cgan2.pth")
torch.save(D.state_dict(), "discriminator_cgan2.pth")
torch.save(clf.state_dict(), "classifier_cgan2.pth")


[Epoch 1] D Loss: 658.8518, G Loss: 2413.1014, C Loss: 1437.5552
[Epoch 2] D Loss: 656.2894, G Loss: 2336.0910, C Loss: 1322.8202
[Epoch 3] D Loss: 652.2607, G Loss: 2153.4269, C Loss: 1275.4349
[Epoch 4] D Loss: 667.2299, G Loss: 2128.5513, C Loss: 944.4518
[Epoch 5] D Loss: 656.3497, G Loss: 2019.6076, C Loss: 699.2260
[Epoch 6] D Loss: 699.9287, G Loss: 1833.4889, C Loss: 591.2645
[Epoch 7] D Loss: 744.1338, G Loss: 1701.0849, C Loss: 484.7723
[Epoch 8] D Loss: 777.9539, G Loss: 1671.4868, C Loss: 459.4634
[Epoch 9] D Loss: 790.6512, G Loss: 1552.3411, C Loss: 456.4172
[Epoch 10] D Loss: 793.9263, G Loss: 1565.2438, C Loss: 438.2249
[Epoch 11] D Loss: 795.9860, G Loss: 1593.8712, C Loss: 442.2619
[Epoch 12] D Loss: 793.4390, G Loss: 1587.5938, C Loss: 400.5894
[Epoch 13] D Loss: 802.1005, G Loss: 1610.5773, C Loss: 370.0211
[Epoch 14] D Loss: 807.5804, G Loss: 1599.9068, C Loss: 347.7442
[Epoch 15] D Loss: 824.2466, G Loss: 1555.1756, C Loss: 332.6735
[Epoch 16] D Loss: 830.7850, G 